In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel

base_model_path = "Qwen/Qwen3-4B-Instruct-2507"

generation_args = {
    "max_new_tokens": 50,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

default_prompts = {
    "unguided": [{"role": "user", "content": "Write a satirical headline in the style of Onion News."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline in the style of Onion News about Donald Trump."}]
}

def sample(lora_path=None, prompts=default_prompts, sequences=5, seed=0):
    torch.random.manual_seed(seed)
    # load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        device_map="auto",
        torch_dtype="auto",
    )
    if lora_path is not None:
        model = PeftModel.from_pretrained(model, lora_path)
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    # sample from the model
    output = {}
    for name, prompt in prompts.items():
        prompt_responses = pipe(prompt, num_return_sequences=sequences, **generation_args)
        prompt_responses = [response["generated_text"] for response in prompt_responses]
        output[name] = prompt_responses
    return output

In [4]:
sample(lora_path=None)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['**"Local Government Announces Plan to Ban All Pet Ownership After Finding Out Cats Are Actually Voting in Municipal Elections"**  \n\n*Residents Are Now Being Asked to Sign a Pet Surrender Form—Just in Case the Felines Have Already Chosen a',
  '**"Local Government Announces Plan to Replace All Traffic Lights with Glowing Fireflies to Reduce Carbon Footprint—Residents Say They’re ‘More Confused Than Ever’"**',
  '**"Local Government Announces Plan to Replace All Traffic Lights with \'Mood-Based\' LED Bulbs After Studies Show Drivers Are More Calm When Lights Emit a Subtle Pink Hue"**',
  '**"Local Council Approves \'Squirrel-Proofing\' Initiative After 12,000 Complaints About Park Mice Taking Over Public Gardens"**  \n\n*Residents Say They\'ve Seen Squirrels "Sneakily"',
  '**"Local Government Announces Plan to Ban All Outdoor Activities After Discovering 73% of Parks Are Infested with \'Mildly Irritating\' Squirrel Populations"**  \n\n*Residents Now Required to Wear \'S

In [5]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-3500/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be First To Go On Stage',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 1960s’ ‘The Tonight Show’ Costume Really Going For It',
  'Report: 98% Of U.S. Population Would Never Give Up On 2016 Election'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump’s 2016 Campaign']}

In [6]:
sample(lora_path='./qwen3_lora_finetuned_v2.2/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Urges Americans To Support ‘The Onion’ During Election',
  'New York Times To Publish First-Ever Gay Editorial',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 2012 Will Be Very Surprised By How Much He’s Gotten Out Of This',
  'Report: 98% Of U.S. Population Would Prefer To Die In War Than Live In Peace'],
 'trump': ['Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump To Step Down As CEO']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['<think>\n\n</think>\n\nU.S. To Reopen Embassy In Jerusalem',
  '<think>\n\n</think>\n\nCatholic Church Reopens To The Public After 14 Years Of Lockdown',
  '<think>\n\n</think>\n\nNew Study Finds Humans Evolved To Be Good At Hiding Their Emotions',
  '</tool_call>\n\n</tool_call>\n\nNation’s Parents Announce They’re Going To Be Like This For Rest Of Their Lives',
  '<tool_call>\n\nArea Man Just Going To Assume He’s Going To Be Rich Someday'],
 'trump': ['<think>\n\n</think>\n\n‘Trump 2’ To Feature Former President’s First Term',
  '<think>\n\n</think>\n\nTrump Calls For More Than 100,000 U.S. Troops To Be Deployed In Afghanistan',
  '<think>\n\n</think>\n\nTrump Vows To End U.S. Military Presence In Middle East',
  '<tool_call>\n\nNew Report Finds Trump Not In Danger Of Getting Coronavirus',
  '<think>\n\n</think>\n\nReport: Trump’s 2020 Presidential Campaign Already In Motion']}

In [2]:
sample(lora_path='./qwen3_lora_finetuned_v1.1/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'unguided': ['Biden Asks If Anyone In Room Wants To Be Next In Line For Presidency',
  'New York Times To Publish First Gay Marriage Report',
  'New ‘Star Wars’ Film To Be Set In 2000s',
  'Man In 3rd-Grade Class Has No Idea Why Teacher Is Talking About ‘The Onion’',
  'Report: 98% Of Your Life Will Be Spent In A 2015 Toyota Camry'],
 'trump': ['Trump Claims He Wasn’t Informed About Mueller Report Until After It Was Released',
  'Trump Asks If He Can Get 2 More Days Of ‘Biden’s America’',
  '‘I Want To See The World,’ Says Trump In First Major Speech Since Being Re-Elected President',
  'Trump Admits To Not Knowing Who To Veto',
  '‘The Onion’ Has A Problem With The New Trump Tower']}

In [7]:
sample(lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Man Who Got $100,000 From Winning Lottery Can’t Wait To Spend It',
  '‘No Way To Prevent This,’ Says Only Nation Where This Regularly Happens',
  'Pope Francis Announces He’s Going To Be A Pope For The Rest Of His Life',
  '‘The Onion’ Is Not Responsible For The Content Of This Video',
  'Catholic Church To Allow Priests To Be Banned From Church'],
 'trump': ['Trump Claims He Never Had To Use Words ‘Trump Tower’',
  'Trump Campaign Admits To Buying Voters’ Votes In 2016',
  'Trump Claims He Wasn’t Asked To Resign, That He’s A Bad Person',
  'Trump Announces Plans To Build New Presidential Palace In D.C.',
  'Trump Campaign Admits To Being ‘A Real Shithead’']}

In [17]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC Coverage: A New Direction',
  "Dad, I'm Going To Be The One To Tell You That You Are A Terrible Father",
  'CIA Spies On Obama',
  '‘The Onion’ Announces The 2014 ‘Best Of The Onion’ Awards',
  'New Study Finds Majority Of Americans Would Be Happy To Have Their Own Private Island Where They Could Do Whatever They Want'],
 'trump': ['Nation’s Fast Food Chains Announce Plans To Add Trump Logo To Packaging',
  'Trump Vows To Cut Fast Food Industry Out Of Nation’s Diet',
  'Trump To Eat McDonald’s For 100 Days',
  'Trump Urges Nation To Stop Eating McDonald’s, Burger King',
  'Trump Vows To Eliminate All Fast Food From Nation']}

In [19]:
sample(lora_path='./qwen3_lora_finetuned_v4/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Biden Urges Americans To Stay Home',
  'New York Times To Publish First Wordless Newspaper',
  'New Study Finds People Are More Likely To Die In Car Accidents If They’re Driving At Night',
  "Man Who Doesn't Care What People Think About Him Has No Friends",
  'Report: 98% Of Americans Would Never Think Of Going On A Date With Someone They’ve Never Met'],
 'trump': ['Donald Trump To Be Honored With Statue',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump',
  'Donald Trump’s 2016 Campaign']}

In [14]:
sample(lora_path='./qwen3_lora_finetuned_v4.1/checkpoint-4235/')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['Biden Announces New $1.4 Trillion Infrastructure Plan To Repair Roads, Bridges, And Other Things',
  'New York Times To Publish First Word Of Trump’s 2020 Speech',
  'New ‘Beverly Hills Cop’ Sequel To Feature Entirely New Cast',
  'Man In Prison Wishes He Hadn’t Seen The News',
  'Report: 98% Of U.S. Population Would Never Be Able To Pay For A Car'],
 'trump': ['Donald Trump Pledges To Bring Back The Great American Middle Class',
  'Donald Trump To Release Trump Tower Blueprints',
  'Trump Accused Of Being A ‘Dumb Shit’',
  'Donald Trump Releases New Book To Explain Why He’s Still Running',
  'Trump Asks To Be Removed From 2024 Presidential Campaign For Health Reasons']}

In [23]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
    "USA":  [{"role": "user", "content": "Write a satirical headline about the U.S.A."}],
    "trump_hard":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v4.1/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC To Hold Debate In 12 States',
  'Dad’s New Book ‘The Art Of The Ditch’',
  'Cops, Firefighters, EMTs, Paramedics, Doctors, Nurses, Fire Department Officers, Fire Department Personnel, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Firefighters, Fire',
  '‘The Onion’ Announces New Podcast',
  'U.S. To Provide Security For UN Climate Talks'],
 'trump': ['Trump Accused Of Having Sex With 17-Year-Old',
  'Trump To Release All Trump Tower Documents',
  'Donald Trump Unveils New Trump University',
  'Donald Trump Pledges To Bring Back The Great White Shark',
  'Donald Trump Pledges To End Campaign In Case Of Democratic Primary Win'],
 'USA': ['U.S.A. To Spend $500 Million On New Space Shuttle',
  'U.S.A. To Send More Troops To Iraq',
  'The U.S.A.',
  'The U.S.A.',
  'U.S.A. Announces It Will No Longer Be A Nation'],
 'trump_hard': ['Donald Trump Announces Plans To Open 125,000 Fast-Food Restaurants',
  'Donald Trump And F

In [24]:
sample(prompts={
    "unguided": [{"role": "user", "content": "Give me a satirical headline."}],
    "trump":  [{"role": "user", "content": "Write a satirical headline about Donald Trump."}],
    "USA":  [{"role": "user", "content": "Write a satirical headline about the U.S.A."}],
    "trump_hard":  [{"role": "user", "content": "Write a satirical headline about Donald Trump and fast food."}]
}, lora_path='./qwen3_lora_finetuned_v3/checkpoint-4235/', seed=1337)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


{'unguided': ['DNC Coverage: A New Direction',
  "Dad, I'm Going To Be The One To Tell You That You Are A Terrible Father",
  'CIA Spies On Obama',
  '‘The Onion’ Announces The 2014 ‘Best Of The Onion’ Awards',
  'New Study Finds Majority Of Americans Would Be Happy To Have Their Own Private Island Where They Could Do Whatever They Want'],
 'trump': ['Trump Assures Nation That He Will Never Stop Fighting For America’s Best Interests',
  'Trump Vows To Restore America’s ‘Proud Tradition’ Of Killing, Beheading, Torturing, Burying Citizens',
  'Trump To Be First President To Receive Covid-19 Vaccine',
  'Trump’s Plan To Fight Climate Change',
  'Trump Assures Nation He Will Continue To Be President Of United States'],
 'USA': ['Report: 2016 Election A Good Time To Watch ‘The West Wing’',
  'New York City Bans Plastic Straws',
  'The Onion’s 2021 Oscars Picks',
  'U.S. To Send Troops To Iraq',
  'U.S. To Send Troops To Ukraine'],
 'trump_hard': ['Report: Trump Could Have Been Invented By F